In [4]:
from glob import glob
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import StratifiedShuffleSplit
import itertools
from collections import defaultdict

all_model_scores = defaultdict(list)

# For every data set in the data repository...
for dataset in glob('data/ecoli.csv.gz'):
    # Read the data set into memory
    input_data = pd.read_csv(dataset, compression='gzip')
    
    for dataset_repeat in range(1, 31):
        # Divide the data set into a training and testing sets, each time with a different RNG seed
        training_indices, testing_indices = next(iter(StratifiedShuffleSplit(input_data['class'].values,
                                                                             n_iter=1,
                                                                             train_size=0.75,
                                                                             test_size=0.25,
                                                                             random_state=dataset_repeat)))
        
        training_features = input_data.loc[training_indices].drop('class', axis=1).values
        training_classes = input_data.loc[training_indices, 'class'].values

        testing_features = input_data.loc[testing_indices].drop('class', axis=1).values
        testing_classes = input_data.loc[testing_indices, 'class'].values
        
        # These are parameters listed in the project document
        max_depth_list = [1, 2, 3, 4, 5, 10, 20, 50, None]
        max_features_list = [0.1, 0.25, 0.5, 0.75, 'sqrt', 'log2', None]
        criterion_list = ['gini', 'entropy']
        
        # For every DecisionTreeClassifier parameter combination...
        for (max_depth, max_features, criterion) in itertools.product(max_depth_list,
                                                                      max_features_list,
                                                                      criterion_list):
            # Create and fit the model on the training data
            clf = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features, criterion=criterion)
            clf.fit(training_features, training_classes)
            
            # Store the model's score with the key (name of data set, model, model parameters)
            all_model_scores[(dataset,
                              'DecisionTreeClassifier',
                              (max_depth, max_features, criterion))].append(clf.score(testing_features,
                                                                                      testing_classes))

In [5]:
import os
import gzip

if not os.path.isdir('benchmark_results'):
    os.mkdir('benchmark_results')

with gzip.open('benchmark_results/DecisionTreeClassifier-benchmarks.tsv.gz', 'wb') as out_file:
    header_text = '\t'.join(['dataset',
                              'model',
                              'parameters',
                              'testing_score']) + '\n'
    out_file.write(header_text.encode('UTF-8'))
    
    for (dataset, model, params) in all_model_scores:
        param_string = ''
        param_string += 'max_depth={},'.format(params[0])
        param_string += 'max_features={},'.format(params[1])
        param_string += 'criterion={}'.format(params[2])
        
        for testing_score in all_model_scores[(dataset, model, params)]:
            out_text = '\t'.join([dataset[5:-7],
                                  model,
                                  param_string,
                                  str(testing_score)]) + '\n'
            out_file.write(out_text.encode('UTF-8'))